# Redes Neurais Artificiais

## Preparando os dados

Vamos criar uma rede neural simples para prever *churn* de clientes. 

Vamos começar pela leitura dos dados:

In [0]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/WoMakersCode/data-science-bootcamp/master/5.1%20Deep%20Learning/data/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [4]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [5]:
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

Para este exemplo vamos fazer um tratamento simples dos dados, apenas convertendo as variáveis categoricas em dummies:

In [6]:
df = pd.get_dummies(df, columns=['Geography', 'Gender'])
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited', 'Geography_France', 'Geography_Germany',
       'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

Vamos separar os dados de teste e treinamento:

In [0]:
X = df[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember','EstimatedSalary', 
        'Geography_France', 'Geography_Germany', 'Geography_Spain', 'Gender_Female']]

y = df['Exited']

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X , y, test_size = 0.1)
X_train, X_val, y_train, y_val = train_test_split(X_train , y_train, test_size = 0.1)

print(X_train.shape)

(8100, 12)


In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_val = sc.transform(X_val)

## Construindo o modelo

Agora com o dados prontos vamos montar a nossa rede neural. Vamos usar a library [Keras](https://keras.io) rodando em cima do [TensorFlow](https://tensorflow.org/)


1. Definição da arquitetura

2. Compilação

3. Treinamento

4. Avaliação

### 1. Definição da arquitetura: 
Definir a arquitetura da rede

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
def build_model():
    model = Sequential()
    
    # primeira camada adiciona o shape do input
    # adiciona a funcao de ativacao
    # quantidade de units (neurônios)
    #input_dim = quantas variáveis de entrada (presentes no X)
    # também é possível alterar a inicializacao, bias, entre outros -- https://keras.io/layers/core/
    model.add(Dense(units=10, input_dim=12, activation='relu'))
    model.add(Dense(10, activation='relu'))
    
    #Camada de saida com o resultado de 1 classe e a ativação sigmoid -- outras funções de ativação: https://keras.io/activations/
    model.add(Dense(1, activation='sigmoid'))
    return model

### Vamos entender melhor as funções de ativação:

Em cada neurônio da rede há uma função de ativação, que decide se o neurônio deve ser *ativado*, e transmitir informações para a próxima camada.

![](https://i1.wp.com/deeplearningbook.com.br/wp-content/uploads/2018/02/act.png?w=406)

A função mais comum nas camadas intermediárias é a relu:

![](https://cdn-images-1.medium.com/max/937/1*oePAhrm74RNnNEolprmTaQ.png)

Na camada de saída a rede precisa nos retornar a probabilidade do cliente fazer o cancelamento.

Por ser uma probabilidade (de 0 a 1), nós usamos a função sigmoid:

![as vezes a função sigmóide é simplesmente representada pela curva S](https://sabedoriararefeita.files.wordpress.com/2016/02/ann_sigmoid.png?w=615)


Outras funções comuns:

Softmax -> Usada na camada de output para problemas de multiclasse, a soma das probabilidades de todas as classes dará 1.

elu -> para ser usada nas camadas intermediarias no lugar da relu, uma exponencial é aplicada nos valores menores que 0.

> Em regressão não há função de ativação na camada de output

outras funções de ativação: https://keras.io/activations/
explicações extras: http://deeplearningbook.com.br/funcao-de-ativacao/

In [12]:
model = build_model()

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                130       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 251
Trainable params: 251
Non-trainable params: 0
_________________________________________________________________


> Podemos ver que na primeira camada 130 parâmetros (pesos) serão aprendidos ((12 inputs x 10 layers) + (1 bias * 10 layers))

### 2. Compilar o modelo:

Definer como a rede irá aprender. Qual o otimizador com os parâmetros de learning rate, função e parametros específicos da função e a loss function.


In [14]:
# outras funções de loss: https://keras.io/losses/
# outros optimizers: https://keras.io/optimizers/
adam = Adam(lr=0.01)
model.compile(loss='binary_crossentropy', 
             optimizer=adam,
             metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Vamos entender como a rede aprende:

Para aprender os parâmetros $w$ e $b$ é preciso uma **função de custo**. Primeiro, vamos definir uma função de perda ou $Loss Function$ de modo que quanto mais próximo da resposta certa, menor seja o valor dessa função:

$L(\hat{y},y)=-(y\log{\hat{y}} + (1-y)\log{(1-\hat{y})})$ (binary_crossentropy)

> Se uma instância tem label 1, então $(1-y)$ é $0$, deixando apenas o lado esquerdo da equação. Pra que ele seja o menor possível, $\hat{y}$ precisa ser o maior possível, no caso o mais próximo de 1. O oposto também se aplica para quando o label é 0.

Com isso, temos a funcão de custo:

$J(w,b)=\frac{1}{m}\sum_{i=1}^{m}L(\hat{y}^i,y^i)$

Dado nosso custo, queremos encontrar $w$ e $b$ que minimize esse custo. Para isso utilizamos o **Gradiente Descendente**. A função de custo é uma funcão convexa, como uma bacia, então o que o gradiente faz é ir descendo o mais rápido possível até chegar no fundo da bacia, no menor ponto, independente do ponto inicial.

![enter image description here](https://blog.paperspace.com/content/images/2018/05/68747470733a2f2f707669676965722e6769746875622e696f2f6d656469612f696d672f70617274312f6772616469656e745f64657363656e742e676966.gif)

Para fazer essa "decida", utilizaremos a derivada do custo e uma taxa de aprendizado ou *learning rate*, da seguinte forma:

A cada iteração do algoritmo temos $w = w - \alpha \frac{\mathrm{d}J}{\mathrm{d}w}$, sendo $\alpha$ o learning rate.

De modo geral, atualizamos w e b a cada iteração, sendo a velocidade controlada pelo learning rate, até chegarmos no ponto mínimo de custo.

**Mas o que é o Adam então?**

Algoritmo de otimização da taxa de aprendizado adaptável que foi projetado especificamente para o treinamento de redes neurais profundas, pode ser usado em vez do procedimento clássico de descida de gradiente estocástico (SGD) para atualizar os pesos da rede de forma iterativa com base nos dados de treinamento.

![](https://cdn-images-1.medium.com/max/1600/1*X9gB3l_Wh5owNPCUsaYQVQ.png)

Mais informações: [artigo original](https://arxiv.org/abs/1412.6980), [post explicativo](https://towardsdatascience.com/adam-latest-trends-in-deep-learning-optimization-6be9a291375c), [outros otimizadores](http://ruder.io/optimizing-gradient-descent/)



> **Importante**: quando estiverem fazendo experimentos com NN, testem com SGD e Adam e com diferentes **LEARNING RATES**.


### 3. Treinamento

In [15]:
model.fit(x=X_train, y=y_train, validation_data=(X_val,y_val), batch_size=16, epochs=10)




Train on 8100 samples, validate on 900 samples
Epoch 1/10





8100/8100 [==============================] - 2s 199us/step - loss: 0.4379 - acc: 0.8146 - val_loss: 0.4329 - val_acc: 0.8111
Epoch 2/10
8100/8100 [==============================] - 1s 87us/step - loss: 0.3778 - acc: 0.8458 - val_loss: 0.3872 - val_acc: 0.8456
Epoch 3/10
8100/8100 [==============================] - 1s 90us/step - loss: 0.3491 - acc: 0.8578 - val_loss: 0.3871 - val_acc: 0.8456
Epoch 4/10
8100/8100 [==============================] - 1s 86us/step - loss: 0.3449 - acc: 0.8568 - val_loss: 0.3861 - val_acc: 0.8333
Epoch 5/10
8100/8100 [==============================] - 1s 92us/step - loss: 0.3423 - acc: 0.8604 - val_loss: 0.3742 - val_acc: 0.8489
Epoch 6/10
8100/8100 [==============================] - 1s 90us/step - loss: 0.3388 - acc: 0.8604 - val_loss: 0.3907 - val_acc: 0.8322
Epoch 7/10
8100/8100 [==============================] - 1s 93us/step - loss: 0.3375 - acc: 0.8625 - val_loss: 0.3798 - val_acc: 0.8400

> Percebemos que só com 10 épocas a rede ainda não tinha convergido, o loss ainda estava caindo, então poderíamos treinar por mais épocas!

Temos dois parâmetros importantes no treinamento:
- Número de épocas: Quantas vezes a rede vai passar por todos as instâncias
- Tamanho do batch: Qual o tamanho do bloco que ela vai usar, ou seja, quantas instâncias por vez passarão pela rede


### 4. Avaliação

In [0]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score

In [18]:
accuracy_score(y_test, y_pred)

0.86

In [19]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[759  40]
 [100 101]]


In [20]:
recall_score(y_test, y_pred)

0.5024875621890548

In [21]:
precision_score(y_test, y_pred)

0.7163120567375887

Perceba que apesar da nossa acurácia ser alta, a qualidade do modelo é ruim com baixa sensibilidade. Vamos retreinar o mesmo modelo, desta vez passando peso para as classes.

In [0]:
model = build_model()
model.compile(loss='binary_crossentropy', 
             optimizer=adam,
             metrics=['accuracy'])



In [23]:
model.fit(x=X_train, y=y_train, validation_data=(X_val,y_val), batch_size=16, epochs=10, class_weight={0:0.2,1:0.8})

Train on 8100 samples, validate on 900 samples
Epoch 1/10
8100/8100 [==============================] - 1s 119us/step - loss: 0.1716 - acc: 0.7417 - val_loss: 0.5037 - val_acc: 0.7556
Epoch 2/10
8100/8100 [==============================] - 1s 82us/step - loss: 0.1542 - acc: 0.7878 - val_loss: 0.5254 - val_acc: 0.7511
Epoch 3/10
8100/8100 [==============================] - 1s 89us/step - loss: 0.1516 - acc: 0.7942 - val_loss: 0.4991 - val_acc: 0.7756
Epoch 4/10
8100/8100 [==============================] - 1s 91us/step - loss: 0.1505 - acc: 0.7984 - val_loss: 0.4507 - val_acc: 0.8100
Epoch 5/10
8100/8100 [==============================] - 1s 89us/step - loss: 0.1492 - acc: 0.8083 - val_loss: 0.5338 - val_acc: 0.7778
Epoch 6/10
8100/8100 [==============================] - 1s 93us/step - loss: 0.1484 - acc: 0.8102 - val_loss: 0.4613 - val_acc: 0.7922
Epoch 7/10
8100/8100 [==============================] - 1s 90us/step - loss: 0.1483 - acc: 0.8098 - val_loss: 0.4992 - val_acc: 0.7633
Epoch 8

In [24]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

cm = confusion_matrix(y_test, y_pred)
print(cm)

[[647 152]
 [ 59 142]]


## Exercício:
Crie uma nova arquitetura de rede para o mesmo dataset. Mude também o learning rate e a quantidade de épocas e compare os resultados


In [0]:
def build_model2():
    model = Sequential()
    
    model.add(Dense(units=11, input_dim=12, activation='relu'))
    model.add(Dense(11, activation='relu'))
    
    #Camada de saida com o resultado de 1 classe e a ativação sigmoid -- outras funções de ativação: https://keras.io/activations/
    model.add(Dense(1, activation='sigmoid'))
    
    
    return model

In [73]:
model2 = build_model2()
model2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 11)                143       
_________________________________________________________________
dense_20 (Dense)             (None, 11)                132       
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 12        
Total params: 287
Trainable params: 287
Non-trainable params: 0
_________________________________________________________________


In [0]:
## TODO defina o learning rate

adam = Adam(lr=0.015)
model2.compile(loss='binary_crossentropy', 
             optimizer=adam,
             metrics=['accuracy'])

In [75]:
## TODO defina o total de épocas
model2.fit(x=X_train, y=y_train, validation_data=(X_val,y_val), batch_size=16, epochs=12, class_weight={0:0.2, 1:0.8})

Train on 8100 samples, validate on 900 samples
Epoch 1/12
8100/8100 [==============================] - 1s 182us/step - loss: 0.1744 - acc: 0.7175 - val_loss: 0.5296 - val_acc: 0.7333
Epoch 2/12
8100/8100 [==============================] - 1s 93us/step - loss: 0.1587 - acc: 0.7595 - val_loss: 0.5070 - val_acc: 0.7300
Epoch 3/12
8100/8100 [==============================] - 1s 103us/step - loss: 0.1569 - acc: 0.7664 - val_loss: 0.4831 - val_acc: 0.7811
Epoch 4/12
8100/8100 [==============================] - 1s 98us/step - loss: 0.1560 - acc: 0.7691 - val_loss: 0.4977 - val_acc: 0.7389
Epoch 5/12
8100/8100 [==============================] - 1s 102us/step - loss: 0.1547 - acc: 0.7762 - val_loss: 0.4987 - val_acc: 0.7411
Epoch 6/12
8100/8100 [==============================] - 1s 100us/step - loss: 0.1529 - acc: 0.7881 - val_loss: 0.4648 - val_acc: 0.7878
Epoch 7/12
8100/8100 [==============================] - 1s 99us/step - loss: 0.1520 - acc: 0.7898 - val_loss: 0.5320 - val_acc: 0.7489
Epoc

In [76]:
y_pred = model2.predict(X_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[645 154]
 [ 57 144]]


In [77]:
accuracy_score(y_test, y_pred)



0.789

In [78]:
precision_score(y_test, y_pred)

0.48322147651006714

In [79]:
recall_score(y_test, y_pred)

0.7164179104477612

## Por que o crescimento de Deep Learning?

<img src="https://kevinzakka.github.io/assets/app_dl/perf_vs_data.png" alt="drawing" width="600"/>

Algoritmos tradicionais tendem a estabilizar a performance apartir de uma certa quantidade de dados, enquanto redes neurais tendem a ficar cada vez melhores quanto mais dados são utilizados para o aprendizado.

Portanto, o principal motivo que faz com que as NN cresçam nos últimos anos é o grande aumento na quantidade de **dados** disponíveis.  Além disso, o poder **computacional** também é muito maior nos dias atuais, principalmente com a utilização de GPU's. O que também permitiu o desenvolvimento de **algoritmos** mais complexos e potentes.


Neural Networks, mais especificamente Deep Learning, tem grande aplicações em datas não-estruturados, como: Imagens, Aúdios e Textos.